In [1]:
%load_ext lab_black

# Standard Imports

In [2]:
import sys

# sys.path.append('/Users/joachim/texjs/lva/IntroSC/ASC-ODE/build/mechsystem')
sys.path.append("../build/mechsystem")

from mass_spring import *

from pythreejs import *
from time import sleep

## Simple Crane with Vibration

In [ ]:
mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

# Base fix
base = mss.add(Fix((0, 0, 0)))

# Tower vertical
tower_masses = []
tower_height = 5
for i in range(1, 6):
    m = mss.add(Mass(2.0, (0, 0, i)))
    tower_masses.append(m)
    if i == 1:
        mss.add(Spring(1.0, 50000, (base, m)))
    else:
        mss.add(Spring(1.0, 50000, (tower_masses[i - 2], m)))

# Horizontal beam at top
beam_length = 5
beam_masses = []
for i in range(1, 6):
    m = mss.add(Mass(1.0, (i, 0, tower_height)))
    beam_masses.append(m)
    if i == 1:
        mss.add(Spring(1.0, 50000, (tower_masses[-1], m)))
    else:
        mss.add(Spring(1.0, 50000, (beam_masses[i - 2], m)))

# Load at end (pendulum)
load = mss.add(Mass(3.0, (beam_length, 0, tower_height - 1)))
mss.add(Spring(1.0, 30000, (beam_masses[-1], load)))

10

In [ ]:
# Masses
masses_vis = [
    Mesh(SphereBufferGeometry(0.2), MeshStandardMaterial(color="red"), position=m.pos)
    for m in mss.masses
]

# Fixed points
fixes_vis = [
    Mesh(SphereBufferGeometry(0.2), MeshStandardMaterial(color="blue"), position=f.pos)
    for f in mss.fixes
]

# Spring segments
def get_spring_segments():
    segs = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        segs.append([pA, pB])
    return segs
springs_geo = LineSegmentsGeometry(positions=get_spring_segments())
springs_vis = LineSegments2(
    LineSegmentsGeometry(positions=get_spring_segments()),
    LineMaterial(linewidth=3, color="cyan"),
)

### Definiton of the View

In [11]:
def create_scene(masses, fixes, springs, width=600, height=400):
    axes = AxesHelper(1)
    camera = PerspectiveCamera(
        position=[0, 10, 0],
        aspect=width / height,
        lookAt=[0, 0, 0],
        up=[0, 0, 1],
    )
    key_light = DirectionalLight(position=[0, 10, 10])
    ambient_light = AmbientLight()
    scene = Scene(
        children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light]
    )
    controller = OrbitControls(controlling=camera)
    renderer = Renderer(
        camera=camera,
        scene=scene,
        controls=[controller],
        width=width,
        height=height,
    )
    return renderer

In [14]:
create_scene(masses_vis, fixes_vis, springs_vis)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(0.0, 10.0, 0.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0,…

In [19]:
for _ in range(20000):
    mss.simulate(0.02, 100)
    for m, vis in zip(mss.masses, masses_vis):
        vis.position = (m.pos[0], m.pos[1], m.pos[2])

    spring_geo = LineSegmentsGeometry(positions=get_spring_segments())
    springs_vis.geometry = spring_geo

    sleep(0.01)

KeyboardInterrupt: 